In [1]:
import numpy as np
import pandas as p
import os

path = os.getcwd()
path = path[:-path[::-1].index('/')]
path_b = path + "source_tables/built/"

In [2]:
#load needed tables
train = p.read_csv(path_b + "train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv(path_b + "events_prep.csv", usecols = [0,1], dtype={"display_id": int, "document_id" : int})
promoted_content = p.read_csv(path_b + "promoted_content_prep.csv", usecols = [0,1,2,3] , dtype = {"ad_id":int, "document_id":int, "campaign_id":int, "advertiser_id":int})

In [3]:
def ctr(df, over, name):
    name = "score_" + name
    to_drop1 = ['ads_on_doc','clicked','uni_chance','clicked_percent','clicked_percent_normalized','likelihood_normalized']
    to_drop2 = ['total','like_mul_total_normalized']
    #add expected chance of ad to be chosen [uni_chance]
    df['uni_chance'] = 1 / df['ads_on_doc']
    #add clicked precentage
    df['clicked_percent'] = df['clicked'] / df['total']
    #add normalized clicked precentage
    df['clicked_percent_normalized'] = (df['clicked'] + 12 * df['clicked_percent'].mean()) / (12 + df['total'])
    #create a likelihood column which shows how strong the actual 
    #clicked percentage is in relation to the uniform chance, added regularization
    #normalized likelihood
    df['likelihood_normalized'] = df['clicked_percent_normalized'] / df['uni_chance']
    #on the way to adding together same pairs with different ad_on_doc
    df['like_mul_total_normalized'] = df['likelihood_normalized'] * df['total']
    #get scores for the pairs ad_document_id document_id grouped over the ad_on_doc
    df = df.drop(to_drop1,axis = 1).groupby(over).sum().reset_index()
    df[name] = df['like_mul_total_normalized'] / df['total']
    df.drop(to_drop2,axis = 1, inplace=True)
    df.sort_values(name,inplace=True, ascending=False)
    return df

In [4]:
print train.head()
print events.head()
print promoted_content.head()

   display_id   ad_id clicked
0           1   42337   False
1           1  139684   False
2           1  144739    True
3           1  156824   False
4           1  279295   False
   display_id  document_id
0           1       379743
1           2      1794259
2           3      1179111
3           4      1777797
4           5       252458
   ad_id  ad_document_id  campaign_id  advertiser_id
0      1            6614            1              7
1      2          471467            2              7
2      3            7692            3              7
3      4          471471            2              7
4      5          471472            2              7


In [5]:
#add number of ads on display
train_ad_count_per_display = train.groupby(['display_id'])['display_id'].agg({'ads_on_doc' : 'count'}).reset_index()
train = train.merge(train_ad_count_per_display, how = 'left', on = 'display_id')
del train_ad_count_per_display

In [6]:
#add the document_id of the display
train = train.merge(events, how = 'left', on = 'display_id')
#add the document_id describing ad
train = train.merge(promoted_content, how = 'left', on = 'ad_id')

In [7]:
print train.head()

   display_id   ad_id clicked  ads_on_doc  document_id  ad_document_id  \
0           1   42337   False           6       379743          938164   
1           1  139684   False           6       379743         1085937   
2           1  144739    True           6       379743         1337362   
3           1  156824   False           6       379743          992370   
4           1  279295   False           6       379743         1670176   

   campaign_id  advertiser_id  
0         5969           1499  
1        17527           2563  
2        18488           2909  
3         7283           1919  
4        27524           1820  


In [8]:
#normalized CTR of ad alone
ad_ctr = train.groupby(['ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
#normalized CTR of ad alone
ad_document_ctr = train.groupby(['ad_document_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
#normalized CTR of document coupled with ad 
document_on_ad_ctr = train.groupby(['document_id','ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
#normalized CTR of document coupled with ads' document
document_on_ad_document_ctr = train.groupby(['document_id','ad_document_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
#normalized CTR of advertiser alone
advertiser_ctr = train.groupby(['advertiser_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
#normalized CTR of campaign alone
campaign_ctr = train.groupby(['campaign_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
#normalized CTR of advertiser coupled with ad
document_on_advertiser_ctr = train.groupby(['advertiser_id','ads_on_doc','document_id']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
#normalized CTR of campaign coupled with ad
document_on_campaign_ctr = train.groupby(['campaign_id','ads_on_doc','document_id']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()

In [9]:
print ad_ctr.head()
print ad_document_ctr.head()
print document_on_ad_ctr.head()
print document_on_ad_document_ctr.head()
print advertiser_ctr.head()
print campaign_ctr.head()
print document_on_advertiser_ctr.head()
print document_on_campaign_ctr.head()

   ad_id  ads_on_doc  total  clicked
0      1           3      1      0.0
1      3           6      1      0.0
2      3           8     88      5.0
3      4           4      8      1.0
4      4           6      5      1.0
   ad_document_id  ads_on_doc  total  clicked
0               1           4      1      0.0
1               1           8      1      0.0
2            2388           2      9      4.0
3            2388           3     15      5.0
4            2388           4    103     16.0
   document_id  ad_id  ads_on_doc  total  clicked
0           12   5475           4      2      0.0
1           12   6821           4      1      0.0
2           12  11283           4      1      0.0
3           12  20922           4      2      1.0
4           12  23595           4      2      0.0
   document_id  ad_document_id  ads_on_doc  total  clicked
0           12          331537           4      1      0.0
1           12          421022           4      2      0.0
2           12          4

In [10]:
ad_ctr = ctr(ad_ctr, ['ad_id'], 'ad')
print ad_ctr.head()

         ad_id  score_ad
14196    39068  4.801153
98529   257136  4.693780
74810   203377  4.244036
66977   184611  4.223643
123254  309026  4.110138


In [11]:
ad_ctr.to_csv(path + 'source_tables/built/ad_ctr.csv', index = False)

In [12]:
ad_document_ctr = ctr(ad_document_ctr, ['ad_document_id'], 'ad_doc')
print ad_document_ctr.head()

       ad_document_id  score_ad_doc
10152          911959      4.802587
8872           866507      3.839586
32659         1439819      3.792161
18043         1112611      3.707800
6032           735282      3.611745


In [13]:
ad_document_ctr.to_csv(path + 'source_tables/built/ad_document_ctr.csv', index = False)

In [14]:
document_on_ad_ctr = ctr(document_on_ad_ctr, ['document_id', 'ad_id'], 'docXad')
print document_on_ad_ctr.head()

         document_id   ad_id  score_docXad
1524032      1035814  263141      8.695825
1403805       961313  234107      7.334829
1404863       961361  132019      7.294726
1402587       961255  234107      6.753868
2749540      1675109  165380      6.725582


In [15]:
document_on_ad_ctr.to_csv(path + 'source_tables/built/document_on_ad_ctr.csv', index=False)

In [16]:
document_on_ad_document_ctr = ctr(document_on_ad_document_ctr, ['document_id', 'ad_document_id'], 'docXad_doc')
print document_on_ad_document_ctr.head()

         document_id  ad_document_id  score_docXad_doc
1412409      1035814         1637653          8.699057
1301828       961313         1585630          7.336971
1302656       961361         1290377          7.295931
1300824       961255         1585630          6.758153
2516629      1675109         1378725          6.728123


In [17]:
document_on_ad_document_ctr.to_csv(path + 'source_tables/built/document_on_ad_document_ctr.csv', index=False)

In [18]:
advertiser_ctr = ctr(advertiser_ctr, ['advertiser_id'], 'adv')
print advertiser_ctr.head()

      advertiser_id  score_adv
1296           1672   3.514151
2394           3085   3.480352
2505           3246   3.299548
2050           2636   3.298055
370             450   3.006866


In [19]:
advertiser_ctr.to_csv(path + 'source_tables/built/advertiser_ctr.csv', index = False)

In [20]:
campaign_ctr = ctr(campaign_ctr, ['campaign_id'], 'camp')
print campaign_ctr.head()

       campaign_id  score_camp
19264        26479    4.709153
13231        18464    4.106349
16919        23516    3.981440
20295        27793    3.854357
12561        17238    3.835266


In [21]:
campaign_ctr.to_csv(path + 'source_tables/built/campaign_ctr.csv', index = False)

In [22]:
document_on_advertiser_ctr = ctr(document_on_advertiser_ctr, ['document_id', 'advertiser_id'], 'docXadv')
print document_on_advertiser_ctr.head()

         document_id  advertiser_id  score_docXadv
1046047       961313           2980       7.343695
1046500       961361           2506       7.235490
1045495       961255           2980       6.771601
1979145      1675109           2007       6.565584
347891        332268           3283       6.411945


In [23]:
document_on_advertiser_ctr.to_csv(path + 'source_tables/built/document_on_advertiser_ctr.csv', index = False)

In [24]:
document_on_campaign_ctr = ctr(document_on_campaign_ctr, ['document_id', 'campaign_id'], 'docXcamp')
print document_on_campaign_ctr.head()

         document_id  campaign_id  score_docXcamp
1234496       961313        25460        7.337785
1235188       961361        16942        7.296388
1233638       961255        25460        6.759781
2391412      1675109        20586        6.729089
2435011      1687168        20586        6.413736


In [25]:
document_on_campaign_ctr.to_csv(path + 'source_tables/built/document_on_campaign_ctr.csv', index = False)